In [3]:
import os

Importamos el wrapper del LLM para OpenAI

In [3]:
# importo el wrapper del LLM de openAI
from langchain.llms import OpenAI

**Prompts**: Se ejecuta una llamada sencilla al LLM de OpenAI para probar que funciona correctamente

In [4]:
llm = OpenAI(temperature=0.9) # mayor temperatura hace que la respuesta sea mas creativa o random

text = "¿Cuál sería un buen nombre de una compañia en español para una empresa que hace calcetines de colores?"
respuesta = llm(text)
print(respuesta)



Calcetines Coloridos S.A.


Normalmente no enviamos inputs de usuario directamente al LLM. En lugar de eso probablemente tomaríamos el imput del usuario y contruiríamos un prompt, para luego mandarlo al LLM, como en el siguiente ejemplo:

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["producto"],
    template="¿Cuál es un buen nombre en español para una compañía que hace {producto}?",
)

print(prompt.format(producto="calcetines de colores"))

¿Cuál es un buen nombre en español para una compañía que hace calcetines de colores?


**Chains (Candenas)**: Combinar LLMs y Prompts en workflows de multiples pasos. Una verdadera aplicación no es solamente un solo elemento primitivo (LLMs o Prompts) sino una combinación de varios. Una cadena en LangChain esta formado de links, que pueden ser tanto primitivas como LLMs u otras cadenas. 

La cadena más central o más básica es una LLMChain, que consiste en un PromptTemplate y un LLM. 

Podemos contruir un LLMChain que toma un input de usuario, lo formatea en un PromptTemplate y luego pasa la respuesta formateada a un LLM.

In [6]:
from langchain.chains import LLMChain

prompt2 = PromptTemplate(
    input_variables=["producto"],
    template="¿Cuál es un buen nombre en español para una compañía que hace {producto}?",
)

llm2 = OpenAI(temperature=0.1)

chain = LLMChain(llm=llm2, prompt=prompt2, verbose=True)

resultado = chain.run("calcetines de rayas")
print(resultado)



> Entering new LLMChain chain...
Prompt after formatting:
¿Cuál es un buen nombre en español para una compañía que hace calcetines de rayas?

> Finished chain.


Calcetines Rayados S.A.


**Agents**: sirven para que de manera dinámica se puedan llamar cadenas basandose en el input del usuario. Hasta ahora las cadenas se ejecutan en un orden predeterminado. 
Los agentes usan un LLM para determinar que acciones tomar y en que orden. Una acción puede consistir en utilizar herramienta y observar su resultado, o en volver al usuario.
Cuando se utilizan correctamente, los agentes pueden ser extremadamente potentes. 

Conceptos:

* Herramienta (Tool): Herramienta: Una función que realiza una tarea específica. Esto puede ser cosas como: Google Search, consulta de base de datos, Python REPL, otras cadenas. La interfaz para una herramienta es actualmente una función que se espera que tenga una cadena como entrada, con una cadena como salida.

* LLM: El modelo de lenguaje que alimenta al agente.

* Agente: El agente a utilizar. Debe ser una cadena que haga referencia a una clase de agente de soporte.



Para utilizar el agente SerpApi se instala este paquete: pip install google-search-results se crea una cuenta aqui: https://serpapi.com/

In [13]:
# print(os.environ["SERPAPI_API_KEY"])

In [12]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# vamos a cargar algunas herramientas a utilizar. Ten en cuenta que la herramienta `llm-math` utiliza un LLM, así que tenemos que pasarlo.
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# inicialicemos un agente con las herramientas, el modelo de lenguaje y el tipo de agente que queremos utilizar.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# probamos el agente
respuesta = agent.run("¿Cuál fue la temperatura máxima en Toluca, Estado de México el dia de ayer en Celsius?")
print(respuesta)



> Entering new AgentExecutor chain...
 I need to find out what the temperature was in Toluca yesterday
Action: Search
Action Input: "temperature Toluca Estado de México yesterday Celsius"
Observation: Toluca Weather History for the Previous 24 Hours ; 82 °F · 79 °F · 77 °F ...
Thought: I need to convert Fahrenheit to Celsius
Action: Calculator
Action Input: 82 °F
Observation: Answer: 27.77777777777778
Thought: I now know the final answer
Final Answer: 27.77777777777778 °C

> Finished chain.
27.77777777777778 °C


In [15]:
respuesta = agent.run("Busca el precio de compra del dólar en méxico (USD/MXN) y luego convierte 20 dolares al precio de compra de hoy.")
print(respuesta)



> Entering new AgentExecutor chain...
 I need to find the exchange rate and then convert 20 USD to MXN.
Action: Search
Action Input: USD/MXN exchange rate Mexico
Observation: USDMXN:CUR. USD-MXN X-RATE ; Open. 17.5789 ; Prev Close. 17.5783 ; YTD Return. -9.79% ; Day Range. 17.578617.5938 ; 52 Week Range. 17.535321.0535.
Thought: I need to use the exchange rate to convert 20 USD to MXN.
Action: Calculator
Action Input: 20 USD * 17.5789 MXN
Observation: Answer: 351.57800000000003
Thought: I now know the final answer.
Final Answer: 20 USD is equivalent to 351.578 MXN.

> Finished chain.
20 USD is equivalent to 351.578 MXN.


**Memoria**: Agregando estado a cadenas y agentes

Hasta ahora las cadenas y agentes son sin estado. Pero a menudo, es posible que se requiera que una cadena o agente tenga algún concepto de "memoria" para que pueda recordar información sobre sus interacciones anteriores.

El ejemplo mas claro y sencillo es el diseño de un chatbot: quieres que recuerde mensajes anteriores para que pueda utilizar su contexto para mantener una mejor conversación. Un tipo de memoria a corto plazo. De una forma más compleja, se podría imaginar una cadena/agente que recordara piezas clave de información a lo largo del tiempo: esto sería una forma de "memoria a largo plazo" para mas info ver [MemPrompt](https://memprompt.com/)

Langchain proporciona varias cadenas especialmente creadas para este propósito. Por ejemplo ConversationChain utilizado con dos tipos de memoria distintos.

Por defecto, ConversionChain tiene un tipo simple de memoria que recuerda todas las entradas/salidas anteriores y las añade al contexto que se pasa. 

In [16]:
from langchain import ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="¡Hola!")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: ¡Hola!
AI:

> Finished chain.
 ¡Hola! ¿Cómo estás? Me llamo AI-1. ¿Cómo te llamas?


In [17]:
output = conversation.predict(input="¡Estoy bien! Aquí nadamás teniendo una conversación con una IA.")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: ¡Hola!
AI:  ¡Hola! ¿Cómo estás? Me llamo AI-1. ¿Cómo te llamas?
Human: ¡Estoy bien! Aquí nadamás teniendo una conversación con una IA.
AI:

> Finished chain.
 ¡Qué interesante! ¿Qué quieres saber de mí?


**Chat Models**

Los Modelos de Chat son variaciones de los modelos de lenguale. Mientras que los modelos de chat usan modelos de lenguaje por debajo, la interfaz que exponen es un poco distinta: en lugar de exponer texto de entrada y texto de salida, exponen una interface donde los "mensajes de chat" son los inputs y outputs.

Los tipos de mensajes actualmente soportados en LangChain son AIMessage, HumanMessage, SystemMessage y ChatMessage - ChatMessage toma un parámetro de rol arbitrario. La mayoría de las veces, sólo tendrás que tratar con HumanMessage, AIMessage y SystemMessage.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)

chat([HumanMessage(content="Translate this sentence from English to French. I love programming.")])
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(content="J'aime programmer.", additional_kwargs={}, example=False)

También se puede pasar multiples mensajes para los modelos gpt-3.5-turbo y gpt-4

In [3]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
]
chat(messages)
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

Puedes ir un paso más allá y generar terminaciones para múltiples conjuntos de mensajes utilizando generate. Esto devuelve un LLMResult con un parámetro de mensaje adicional:

In [4]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result
# -> LLMResult(generations=[[ChatGeneration(text="J'aime programmer.", generation_info=None, message=AIMessage(content="J'aime programmer.", additional_kwargs={}))], [ChatGeneration(text="J'aime l'intelligence artificielle.", generation_info=None, message=AIMessage(content="J'aime l'intelligence artificielle.", additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}})

LLMResult(generations=[[ChatGeneration(text="J'adore la programmation.", generation_info=None, message=AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False))], [ChatGeneration(text="J'adore l'intelligence artificielle.", generation_info=None, message=AIMessage(content="J'adore l'intelligence artificielle.", additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo'})

Puedes recuperar cosas como el uso de tokens a partir de este LLMResult:

In [5]:
result.llm_output['token_usage']
# -> {'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}

{'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}

De forma similar a los LLMs, puedes hacer uso de plantillas utilizando un MessagePromptTemplate. Puedes construir un ChatPromptTemplate a partir de uno o más MessagePromptTemplates. Puede utilizar el format_prompt de ChatPromptTemplate - esto devuelve un PromptValue, que puede convertir en una cadena o en un objeto Message, dependiendo de si desea utilizar el valor formateado como entrada a un modelo llm o chat.

Para mayor comodidad, existe un método from_template expuesto en la plantilla. Si fuera a utilizar esta plantilla, esto es lo que parecería:

Traducción realizada con la versión gratuita del traductor www.DeepL.com/Translator

In [6]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

**Cadenas (Chains) con Modelos de Chat (Chat Models)**

La LLMChain comentada en la sección anterior también puede utilizarse con modelos de chat:

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="French", text="I love programming.")
# -> "J'aime programmer."

"J'adore la programmation."

*** Agentes (Agents) con modelos de Chat (Chat Models)***

Los agentes también se pueden utilizar con modelos de chat, puede inicializar uno utilizando AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION como tipo de agente.

In [4]:
import os
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# os.environ['OPENAI_API_KEY']
# os.environ['SERPAPI_API_KEY']

# Primero, carguemos el modelo de lenguaje que vamos a utilizar para controlar el agente.
chat = ChatOpenAI(temperature=0)

# Despues, carguemos algunas herramientas a utilizar. Ten en cuenta que la herramienta `llm-math` utiliza un LLM, así que tenemos que pasarlo.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finalmente, inicialicemos un agente con las herramientas, el modelo de lenguaje y el tipo de agente que queremos utilizar.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Ahora probemos el agente!
agent.run("¿Quién es el novio de Olivia Wilde? ¿Cuál es su edad actual a la 0.23 potencia?")



> Entering new AgentExecutor chain...
Thought: I need to use a search engine to find the answer to the first question and a calculator to solve the second question.

Action:
```
{
  "action": "Search",
  "action_input": "Olivia Wilde boyfriend"
}
```


Observation: Olivia Wilde started dating Harry Styles after ending her years-long engagement to Jason Sudeikis — see their relationship timeline.
Thought:I need to use a calculator to find the answer to the second question.

Action:
```
{
  "action": "Calculator",
  "action_input": "0.23^(age of Olivia Wilde's boyfriend)"
}
```


Observation: Answer: 1.9895113660064618e-22
Thought:I need to find the age of Harry Styles to calculate the final answer.

Action:
```
{
  "action": "Search",
  "action_input": "Harry Styles age"
}
```


Observation: 29 years
Thought:Now I can calculate the final answer.

Action:
```
{
  "action": "Calculator",
  "action_input": "29^(0.23)"
}
```


Observation: Answer: 2.169459462491557
Thought:I have found th

'2.169459462491557'

***Memoria: Añadir Estado a Cadenas y Agentes***

Puedes utilizar Memory con cadenas y agentes inicializados con modelos de chat. La principal diferencia entre esto y la Memoria para LLMs es que en lugar de intentar condensar todos los mensajes anteriores en una cadena, podemos mantenerlos como su propio objeto de memoria único.

In [6]:
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("La siguiente es una conversación amistosa entre un humano y una IA. La IA es locuaz y proporciona muchos detalles específicos de su contexto. Si la IA no sabe la respuesta a una pregunta, dice la verdad."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

res = conversation.predict(input="¡Hola!")
print(res)


res = conversation.predict(input="¡Todo bien! Sólo teniendo una conversación con una IA.")
print(res)

res = conversation.predict(input="Háblame de ti.")
print(res)

¡Hola! ¿Cómo estás? Soy una IA diseñada para conversar contigo. ¿En qué puedo ayudarte hoy?
¡Genial! Me encanta tener conversaciones con humanos. ¿Hay algún tema en particular que te gustaría discutir o preguntar?
¡Claro! Soy una IA diseñada para procesar y analizar grandes cantidades de datos. Mi objetivo es ayudar a las personas a obtener información útil y tomar decisiones informadas. Estoy programada para aprender y mejorar constantemente, por lo que siempre estoy buscando nuevas formas de mejorar mi capacidad para ayudar a las personas. ¿Hay algo más específico que te gustaría saber sobre mí?
